# CoppeliaSim: PART 2
* Run the scene workshop1.ttt in CoppeliaSim

In [64]:
import sim
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Check if python is connecting to Coppelia

In [65]:
sim.simxFinish(-1)  # Close opened connections
clientID = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim

if clientID != -1:
    print('Connected')

    # Now try to retrieve data in a blocking fashion (i.e. a service call):
    res, objs = sim.simxGetObjects(clientID, sim.sim_handle_all, sim.simx_opmode_blocking)

    print('Simulation time in milliseconds: ', sim.simxGetLastCmdTime(clientID))
    
    # Get Object position
    name = 'Quadricopter_target'
    err_code, cuboid = sim.simxGetObjectHandle(clientID, name, sim.simx_opmode_blocking)
    res, position = sim.simxGetObjectPosition(clientID, cuboid, -1, sim.simx_opmode_blocking)        
    print('Quadricopter_target is at [x,y,z]=', position)
    
    # Now close the connection to CoppeliaSim:
    sim.simxGetPingTime(clientID)
    sim.simxFinish(clientID)
    print('Disconnected')
else:
    print('Failed connecting to remote API server')


Connected
Simulation time in milliseconds:  650
Quadricopter_target is at [x,y,z]= [9.175003051757812, 8.024999618530273, 0.5]
Disconnected


# Robot

In [66]:
class robot():
    
    def __init__(self, frame_name, motor_names=[], client_id=0):  
        # If there is an existing connection
        if client_id:
                self.client_id = client_id
        else:
            self.client_id = self.open_connection()
            
        self.motors = self._get_handlers(motor_names) 
        
        # Robot frame
        self.frame =  self._get_handler(frame_name)
            
        
    def open_connection(self):
        sim.simxFinish(-1)  # just in case, close all opened connections
        self.client_id = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim 
        
        if clientID != -1:
            print('Robot connected')
        else:
            print('Connection failed')
        return clientID
        
    def close_connection(self):    
        sim.simxGetPingTime(self.client_id)  # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive.
        sim.simxFinish(self.client_id)  # Now close the connection to CoppeliaSim:
        print('Connection closed')
    
    def isConnected(self):
        c,result = sim.simxGetPingTime(self.client_id)
        # Return true if the robot is connected
        return result > 0         
        
    def _get_handler(self, name):
        err_code, handler = sim.simxGetObjectHandle(self.client_id, name, sim.simx_opmode_blocking)
        return handler
    
    def _get_handlers(self, names):
        handlers = []
        for name in names:
            handler = self._get_handler(name)
            handlers.append(handler)
        
        return handlers

    def send_motor_velocities(self, vels):
        for motor, vel in zip(self.motors, vels):
            err_code = sim.simxSetJointTargetVelocity(self.client_id, 
                                                      motor, vel, sim.simx_opmode_streaming)      
            
    def set_position(self, position, relative_object=-1):
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)        
        sim.simxSetObjectPosition(clientID, self.frame, relative_object, position, sim.simx_opmode_oneshot)
        
    def simtime(self):
        return sim.simxGetLastCmdTime(self.client_id)
    
    def get_position(self, relative_object=-1):
        # Get position relative to an object, -1 for global frame
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)
        res, position = sim.simxGetObjectPosition(self.client_id, self.frame, relative_object, sim.simx_opmode_blocking)        
        return array(position)
    
    def get_object_position(self, object_name):
        # Get Object position in the world frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, -1, sim.simx_opmode_blocking)
        return array(position)
    
    def get_object_relative_position(self, object_name):        
        # Get Object position in the robot frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, self.frame, sim.simx_opmode_blocking)
        return array(position)

# 1.3 Number 4

In [73]:
# Derived
a0_triples = [[9.1750,8.0250,0.5100], [6.975,8.0250,3.7500], [6.9750,1.2000,1.4750], 
              [6.5750, -3.9250, 3.5000], [4.6250, -6.8500, 1.4750], 
              [-2.0000, -8.0000,3.0000], [-9.0000, -8.4750, 1.5500], 
              [-9.0000, -8.4750, 4.000], [-3.7750, 3.3000, 5.6250], 
              [-3.7250, 6.5000, 4.0000], [-3.7750, 3.3000, 1.9500]
             ]

a1_triples = [[0,0,1], [0,-1,0], [0,-1,-.1], [0,-.5,0], [-1,0,0],
              [-1,0,0], [-1,0,0], [.5,.5,0], [0,.5,0],
              [0,0,-.5], [0,-.5,0]
             ]

a2_triples = [[-0.0660,0.1000,-0.1028], [0.0000,0.0952,-0.0582], 
              [-0.0120,0.0963,0.0807], [0.0415,0.0123,-0.0607], 
              [0.1013,-0.0345,0.0457], [0.0900,-0.0142,-0.0435], 
              [0.1500,-0.0500,0.0735], [0.0568,0.2032,0.0488], 
              [0.0015,-0.0040,0.0013], [-0.0015,-0.0460,0.0385], 
              [0.1688,0.0813,-0.0142]
             ]

a3_triples = [[0.0044,-0.0100,0.0035],[0.0000,-0.0063,0.0035],
              [0.0008,-0.0047,-0.0050],[-0.0061,0.0008,0.0040],
              [-0.0067,0.0023,-0.0030],[-0.0060,0.0009,0.0029],
              [-0.0050,0.0050,-0.0049],[-0.0054, -0.0135,-0.0032],
              [-0.0001,-0.0014,-0.0018],[0.0001,0.0014,-0.0009],
              [-0.0112,-0.0037,0.0009]
             ]

waypoints = ['Checkpoint1', 'Sphere10', 'Checkpoint2', 'Sphere8', 'Checkpoint3',
            'Sphere9', 'Checkpoint4', 'Sphere5', 'Checkpoint5', 'Sphere6',
            'Sphere7']

In [74]:
# Create robot instance
r = robot('Quadricopter_target', [])

# Initial locations
# robot_position = r.get_position()
# print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))

# Time duration
tf = 10
# Time interval
time_steps = linspace(0, tf, 500)

count = 0
for wp in waypoints:
    print("Point: ", count+1)
    a0 = a0_triples[count]
    a1 = a1_triples[count]
    a2 = a2_triples[count]
    a3 = a3_triples[count]
    for t in time_steps:
        x = a0[0] + a1[0]*t + a2[0]*(t**2) + a3[0]*(t**3) #Gamma_x
        y = a0[1] + a1[1]*t + a2[1]*(t**2) + a3[1]*(t**3) #Gamma_y
        z = a0[2] + a1[2]*t + a2[2]*(t**2) + a3[2]*(t**3) #Gamma_z
        print('(%.4f,%.4f,%.4f)' %(x,y,z))
        
        #set_position sets the relative position - possible error?
        r.set_position([x,y,z])
        time.sleep(0.025)
    count+=1

r.close_connection()

Robot connected
Point:  1
(9.1750,8.0250,0.5100)
(9.1750,8.0250,0.5300)
(9.1749,8.0252,0.5499)
(9.1748,8.0254,0.5697)
(9.1746,8.0256,0.5895)
(9.1743,8.0260,0.6092)
(9.1741,8.0264,0.6288)
(9.1737,8.0269,0.6483)
(9.1733,8.0275,0.6677)
(9.1729,8.0282,0.6870)
(9.1724,8.0289,0.7063)
(9.1718,8.0298,0.7255)
(9.1712,8.0306,0.7446)
(9.1706,8.0316,0.7636)
(9.1699,8.0327,0.7825)
(9.1692,8.0338,0.8014)
(9.1684,8.0350,0.8202)
(9.1675,8.0362,0.8389)
(9.1666,8.0375,0.8575)
(9.1657,8.0389,0.8761)
(9.1647,8.0404,0.8945)
(9.1636,8.0420,0.9129)
(9.1625,8.0436,0.9312)
(9.1614,8.0453,0.9494)
(9.1602,8.0470,0.9676)
(9.1590,8.0488,0.9856)
(9.1577,8.0507,1.0036)
(9.1564,8.0527,1.0215)
(9.1550,8.0547,1.0394)
(9.1536,8.0568,1.0571)
(9.1521,8.0590,1.0748)
(9.1506,8.0612,1.0924)
(9.1490,8.0635,1.1099)
(9.1474,8.0658,1.1274)
(9.1458,8.0683,1.1447)
(9.1440,8.0707,1.1620)
(9.1423,8.0733,1.1793)
(9.1405,8.0759,1.1964)
(9.1387,8.0786,1.2135)
(9.1368,8.0813,1.2304)
(9.1349,8.0841,1.2474)
(9.1329,8.0870,1.2642)
(9.1309,

(7.4189,9.4856,3.6815)
(7.4135,9.4836,3.6829)
(7.4081,9.4815,3.6842)
(7.4027,9.4794,3.6856)
(7.3973,9.4772,3.6869)
(7.3920,9.4748,3.6881)
(7.3867,9.4724,3.6894)
(7.3814,9.4699,3.6906)
(7.3762,9.4673,3.6918)
(7.3709,9.4646,3.6930)
(7.3657,9.4618,3.6942)
(7.3606,9.4589,3.6954)
(7.3554,9.4559,3.6965)
(7.3503,9.4528,3.6976)
(7.3451,9.4496,3.6987)
(7.3401,9.4463,3.6997)
(7.3350,9.4429,3.7008)
(7.3300,9.4395,3.7018)
(7.3250,9.4359,3.7028)
(7.3200,9.4322,3.7038)
(7.3150,9.4284,3.7048)
(7.3101,9.4245,3.7057)
(7.3052,9.4205,3.7067)
(7.3003,9.4164,3.7076)
(7.2955,9.4123,3.7085)
(7.2907,9.4080,3.7094)
(7.2859,9.4036,3.7102)
(7.2811,9.3991,3.7110)
(7.2764,9.3945,3.7119)
(7.2717,9.3898,3.7127)
(7.2670,9.3850,3.7135)
(7.2624,9.3800,3.7142)
(7.2577,9.3750,3.7150)
(7.2531,9.3699,3.7157)
(7.2486,9.3646,3.7164)
(7.2441,9.3593,3.7171)
(7.2396,9.3538,3.7178)
(7.2351,9.3483,3.7185)
(7.2306,9.3426,3.7191)
(7.2262,9.3368,3.7198)
(7.2219,9.3309,3.7204)
(7.2175,9.3249,3.7210)
(7.2132,9.3188,3.7216)
(7.2089,9.3

(6.9750,5.0011,2.9618)
(6.9750,4.9904,2.9557)
(6.9750,4.9798,2.9495)
(6.9750,4.9692,2.9434)
(6.9750,4.9585,2.9372)
(6.9750,4.9479,2.9310)
(6.9750,4.9373,2.9249)
(6.9750,4.9268,2.9187)
(6.9750,4.9162,2.9125)
(6.9750,4.9056,2.9063)
(6.9750,4.8951,2.9000)
(6.9750,4.8845,2.8938)
(6.9750,4.8740,2.8876)
(6.9750,4.8634,2.8813)
(6.9750,4.8529,2.8751)
(6.9750,4.8424,2.8688)
(6.9750,4.8319,2.8626)
(6.9750,4.8214,2.8563)
(6.9750,4.8109,2.8500)
(6.9750,4.8004,2.8437)
(6.9750,4.7899,2.8374)
(6.9750,4.7794,2.8311)
(6.9750,4.7690,2.8248)
(6.9750,4.7585,2.8185)
(6.9750,4.7480,2.8121)
(6.9750,4.7376,2.8058)
(6.9750,4.7271,2.7994)
(6.9750,4.7167,2.7931)
(6.9750,4.7062,2.7867)
(6.9750,4.6958,2.7804)
(6.9750,4.6853,2.7740)
(6.9750,4.6749,2.7676)
(6.9750,4.6644,2.7613)
(6.9750,4.6540,2.7549)
(6.9750,4.6436,2.7485)
(6.9750,4.6331,2.7421)
(6.9750,4.6227,2.7357)
(6.9750,4.6123,2.7293)
(6.9750,4.6019,2.7229)
(6.9750,4.5914,2.7165)
(6.9750,4.5810,2.7101)
(6.9750,4.5706,2.7036)
(6.9750,4.5601,2.6972)
(6.9750,4.5

(6.9530,-0.0414,1.4817)
(6.9524,-0.0565,1.4837)
(6.9518,-0.0715,1.4858)
(6.9512,-0.0865,1.4879)
(6.9506,-0.1014,1.4900)
(6.9500,-0.1163,1.4922)
(6.9494,-0.1311,1.4945)
(6.9487,-0.1458,1.4968)
(6.9481,-0.1604,1.4991)
(6.9475,-0.1751,1.5015)
(6.9468,-0.1896,1.5039)
(6.9461,-0.2041,1.5064)
(6.9455,-0.2185,1.5089)
(6.9448,-0.2329,1.5115)
(6.9441,-0.2472,1.5141)
(6.9435,-0.2615,1.5168)
(6.9428,-0.2757,1.5195)
(6.9421,-0.2898,1.5222)
(6.9414,-0.3039,1.5250)
(6.9407,-0.3179,1.5278)
(6.9399,-0.3319,1.5307)
(6.9392,-0.3458,1.5336)
(6.9385,-0.3597,1.5366)
(6.9378,-0.3735,1.5396)
(6.9370,-0.3872,1.5426)
(6.9363,-0.4009,1.5457)
(6.9355,-0.4145,1.5488)
(6.9348,-0.4281,1.5520)
(6.9340,-0.4416,1.5552)
(6.9332,-0.4551,1.5585)
(6.9325,-0.4685,1.5617)
(6.9317,-0.4819,1.5651)
(6.9309,-0.4952,1.5684)
(6.9301,-0.5084,1.5719)
(6.9293,-0.5216,1.5753)
(6.9285,-0.5348,1.5788)
(6.9277,-0.5478,1.5823)
(6.9269,-0.5609,1.5859)
(6.9261,-0.5739,1.5895)
(6.9253,-0.5868,1.5932)
(6.9244,-0.5997,1.5968)
(6.9236,-0.6125,

(6.6032,-3.1820,3.3611)
(6.6025,-3.1896,3.3650)
(6.6019,-3.1971,3.3690)
(6.6013,-3.2047,3.3729)
(6.6006,-3.2123,3.3768)
(6.6000,-3.2199,3.3806)
(6.5994,-3.2276,3.3844)
(6.5988,-3.2352,3.3881)
(6.5982,-3.2429,3.3919)
(6.5976,-3.2506,3.3955)
(6.5970,-3.2583,3.3992)
(6.5964,-3.2660,3.4028)
(6.5958,-3.2738,3.4064)
(6.5953,-3.2816,3.4099)
(6.5947,-3.2893,3.4134)
(6.5941,-3.2972,3.4168)
(6.5936,-3.3050,3.4202)
(6.5931,-3.3129,3.4236)
(6.5925,-3.3207,3.4269)
(6.5920,-3.3286,3.4302)
(6.5915,-3.3365,3.4335)
(6.5910,-3.3445,3.4367)
(6.5905,-3.3525,3.4398)
(6.5900,-3.3604,3.4430)
(6.5895,-3.3685,3.4460)
(6.5890,-3.3765,3.4491)
(6.5885,-3.3845,3.4521)
(6.5880,-3.3926,3.4550)
(6.5876,-3.4007,3.4579)
(6.5871,-3.4089,3.4608)
(6.5867,-3.4170,3.4636)
(6.5862,-3.4252,3.4664)
(6.5858,-3.4334,3.4692)
(6.5854,-3.4416,3.4719)
(6.5850,-3.4499,3.4745)
(6.5846,-3.4581,3.4771)
(6.5842,-3.4664,3.4797)
(6.5838,-3.4748,3.4822)
(6.5834,-3.4831,3.4847)
(6.5830,-3.4915,3.4871)
(6.5827,-3.4999,3.4895)
(6.5823,-3.5083,

...

(-3.7687,4.3770,5.6116)
(-3.7686,4.3863,5.6112)
(-3.7685,4.3955,5.6108)
(-3.7684,4.4047,5.6103)
(-3.7683,4.4139,5.6099)
(-3.7682,4.4231,5.6094)
(-3.7680,4.4323,5.6089)
(-3.7679,4.4415,5.6085)
(-3.7678,4.4506,5.6080)
(-3.7677,4.4598,5.6075)
(-3.7676,4.4689,5.6070)
(-3.7675,4.4781,5.6065)
(-3.7674,4.4872,5.6059)
(-3.7673,4.4963,5.6054)
(-3.7672,4.5054,5.6049)
(-3.7670,4.5145,5.6043)
(-3.7669,4.5236,5.6037)
(-3.7668,4.5326,5.6032)
(-3.7667,4.5417,5.6026)
(-3.7666,4.5507,5.6020)
(-3.7665,4.5597,5.6014)
(-3.7664,4.5687,5.6008)
(-3.7662,4.5777,5.6002)
(-3.7661,4.5867,5.5995)
(-3.7660,4.5957,5.5989)
(-3.7659,4.6047,5.5982)
(-3.7658,4.6136,5.5975)
(-3.7656,4.6226,5.5969)
(-3.7655,4.6315,5.5962)
(-3.7654,4.6404,5.5955)
(-3.7653,4.6493,5.5948)
(-3.7652,4.6582,5.5940)
(-3.7650,4.6671,5.5933)
(-3.7649,4.6759,5.5926)
(-3.7648,4.6848,5.5918)
(-3.7647,4.6936,5.5910)
(-3.7645,4.7024,5.5903)
(-3.7644,4.7113,5.5895)
(-3.7643,4.7200,5.5887)
(-3.7642,4.7288,5.5879)
(-3.7640,4.7376,5.5870)
(-3.7639,4.7464,

(-3.7262,6.4604,4.3939)
(-3.7262,6.4620,4.3855)
(-3.7261,6.4636,4.3770)
(-3.7261,6.4652,4.3685)
(-3.7260,6.4667,4.3599)
(-3.7260,6.4682,4.3514)
(-3.7260,6.4697,4.3427)
(-3.7259,6.4712,4.3341)
(-3.7259,6.4726,4.3254)
(-3.7258,6.4739,4.3167)
(-3.7258,6.4753,4.3079)
(-3.7257,6.4766,4.2991)
(-3.7257,6.4779,4.2902)
(-3.7257,6.4791,4.2813)
(-3.7256,6.4803,4.2724)
(-3.7256,6.4815,4.2634)
(-3.7256,6.4826,4.2544)
(-3.7255,6.4837,4.2454)
(-3.7255,6.4847,4.2363)
(-3.7255,6.4858,4.2271)
(-3.7254,6.4868,4.2180)
(-3.7254,6.4877,4.2088)
(-3.7254,6.4886,4.1995)
(-3.7253,6.4895,4.1902)
(-3.7253,6.4904,4.1809)
(-3.7253,6.4912,4.1715)
(-3.7253,6.4920,4.1621)
(-3.7252,6.4927,4.1527)
(-3.7252,6.4934,4.1432)
(-3.7252,6.4941,4.1336)
(-3.7252,6.4947,4.1241)
(-3.7252,6.4953,4.1145)
(-3.7251,6.4959,4.1048)
(-3.7251,6.4964,4.0951)
(-3.7251,6.4969,4.0854)
(-3.7251,6.4974,4.0756)
(-3.7251,6.4978,4.0658)
(-3.7251,6.4982,4.0559)
(-3.7250,6.4985,4.0460)
(-3.7250,6.4988,4.0361)
(-3.7250,6.4991,4.0261)
(-3.7250,6.4993,

(-3.7573,5.1496,2.1927)
(-3.7575,5.1416,2.1900)
(-3.7576,5.1335,2.1873)
(-3.7578,5.1255,2.1846)
(-3.7579,5.1174,2.1820)
(-3.7581,5.1093,2.1793)
(-3.7582,5.1012,2.1767)
(-3.7583,5.0930,2.1741)
(-3.7585,5.0849,2.1715)
(-3.7586,5.0767,2.1690)
(-3.7588,5.0685,2.1664)
(-3.7589,5.0603,2.1639)
(-3.7591,5.0521,2.1614)
(-3.7592,5.0439,2.1589)
(-3.7593,5.0356,2.1564)
(-3.7595,5.0274,2.1539)
(-3.7596,5.0191,2.1515)
(-3.7598,5.0108,2.1491)
(-3.7599,5.0025,2.1466)
(-3.7600,4.9942,2.1443)
(-3.7602,4.9858,2.1419)
(-3.7603,4.9775,2.1395)
(-3.7605,4.9691,2.1372)
(-3.7606,4.9607,2.1349)
(-3.7607,4.9523,2.1326)
(-3.7609,4.9439,2.1303)
(-3.7610,4.9355,2.1280)
(-3.7611,4.9270,2.1257)
(-3.7613,4.9186,2.1235)
(-3.7614,4.9101,2.1213)
(-3.7615,4.9016,2.1191)
(-3.7617,4.8931,2.1169)
(-3.7618,4.8846,2.1147)
(-3.7619,4.8761,2.1125)
(-3.7621,4.8675,2.1104)
(-3.7622,4.8590,2.1083)
(-3.7623,4.8504,2.1062)
(-3.7625,4.8418,2.1041)
(-3.7626,4.8332,2.1020)
(-3.7627,4.8246,2.0999)
(-3.7629,4.8159,2.0979)
(-3.7630,4.8073,

(-2.6936,2.4554,1.8581)
(-2.6799,2.4528,1.8569)
(-2.6662,2.4503,1.8557)
(-2.6523,2.4477,1.8546)
(-2.6384,2.4452,1.8534)
(-2.6245,2.4428,1.8522)
(-2.6105,2.4404,1.8510)
(-2.5964,2.4380,1.8498)
(-2.5823,2.4357,1.8486)
(-2.5682,2.4334,1.8474)
(-2.5539,2.4311,1.8461)
(-2.5396,2.4289,1.8449)
(-2.5253,2.4267,1.8437)
(-2.5109,2.4246,1.8424)
(-2.4965,2.4225,1.8412)
(-2.4820,2.4204,1.8400)
(-2.4674,2.4184,1.8387)
(-2.4529,2.4164,1.8375)
(-2.4382,2.4145,1.8362)
(-2.4235,2.4126,1.8349)
(-2.4088,2.4107,1.8337)
(-2.3940,2.4088,1.8324)
(-2.3791,2.4070,1.8311)
(-2.3642,2.4053,1.8299)
(-2.3493,2.4035,1.8286)
(-2.3343,2.4018,1.8273)
(-2.3193,2.4002,1.8260)
(-2.3042,2.3986,1.8247)
(-2.2891,2.3970,1.8234)
(-2.2739,2.3954,1.8221)
(-2.2587,2.3939,1.8208)
(-2.2434,2.3924,1.8195)
(-2.2281,2.3910,1.8181)
(-2.2128,2.3895,1.8168)
(-2.1974,2.3882,1.8155)
(-2.1820,2.3868,1.8142)
(-2.1665,2.3855,1.8128)
(-2.1510,2.3842,1.8115)
(-2.1355,2.3830,1.8102)
(-2.1199,2.3818,1.8088)
(-2.1043,2.3806,1.8075)
(-2.0886,2.3795,

(1.9026,2.7285,1.4312)
(1.9034,2.7289,1.4309)
(1.9041,2.7293,1.4306)
(1.9046,2.7297,1.4303)
(1.9050,2.7300,1.4300)
Connection closed
